In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('normalize_input424.csv')
df2 = pd.read_csv('USvideos.csv')
category_ID = df2['category_id']
df.insert(2,'category_id',category_ID)
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns = ['video_id'])
df = df.drop(columns = ['daily_views'])
df
#df = df.loc[df['daily_view_classification']== 'Large']
#df = df.drop(columns = ['daily_view_classification'])
#daily_view_classification: Large, Middle, Small

,category_id,sub_date,channel frequency,category_of_title_length,tags_length_category,description_contains_facebook,description_contains_instagram,title_contains_num,title_contains_special,title_contains_brackets,has_description,title_length_normal,tags_length_normal,daily_likes_normal,daily_dislikes_normal,daily_comment_count_normal,stop_words_count_title_normal,stop_words_count_tags_normal,hour_normal,daily_view_classification
0,22,2,high,low,low,1,1,0,0,0,0,0.260870,0.000000,0.014826,0.004174,0.021754,0.307692,0.000000,0.739130,Large
1,24,2,high,moderate,low,0,0,0,1,1,0,0.391304,0.044118,0.025047,0.008650,0.017320,0.230769,0.029412,0.304348,Large
2,23,3,high,moderate,moderate,1,1,0,1,0,0,0.391304,0.323529,0.025091,0.005008,0.007437,0.000000,0.058824,0.826087,Large
3,24,2,high,low,high,1,1,0,1,0,0,0.173913,0.382353,0.002622,0.000937,0.002926,0.076923,0.117647,0.478261,Middle
4,24,3,high,low,moderate,1,1,0,1,0,0,0.173913,0.191176,0.022720,0.001866,0.015924,0.076923,0.000000,0.782609,Large
5,28,2,high,low,low,1,1,1,0,0,0,0.173913,0.088235,0.002516,0.000718,0.001955,0.076923,0.029412,0.826087,Middle
6,24,3,high,moderate,high,1,1,0,1,0,0,0.347826,0.602941,0.002748,0.002294,0.001789,0.000000,0.205882,0.217391,Large
7,28,3,high,moderate,low,1,1,1,0,0,0,0.304348,0.176471,0.004065,0.000729,0.003120,0.230769,0.176471,0.913043,Large
8,1,2,high,high,high,0,0,1,1,0,0,0.478261,0.397059,0.000913,0.000166,0.000464,0.076923,0.088235,0.608696,Large
9,25,2,high,high,moderate,0,0,0,1,0,0,0.478261,0.279412,0.003261,0.001917,0.003229,0.461538,0.088235,0.565217,Middle


In [35]:
df_large = df.loc[df['daily_view_classification']== 'Large']
df_middle = df.loc[df['daily_view_classification']== 'Middle']
df_small = df.loc[df['daily_view_classification']== 'Small']

In [36]:
df1 = df_large.sample(frac = 0.005)
df2 = df_middle.sample(frac = 0.005)
df3 = df_small.sample(frac = 0.005)

In [37]:
frames = pd.concat([df1, df2,df3], axis=0, sort=False)

In [38]:
from sklearn.utils import shuffle
frames = shuffle(frames)

In [39]:
frames

,category_id,sub_date,channel frequency,category_of_title_length,tags_length_category,description_contains_facebook,description_contains_instagram,title_contains_num,title_contains_special,title_contains_brackets,has_description,title_length_normal,tags_length_normal,daily_likes_normal,daily_dislikes_normal,daily_comment_count_normal,stop_words_count_title_normal,stop_words_count_tags_normal,hour_normal,daily_view_classification
21468,25,5,high,moderate,low,0,0,1,1,0,0,0.304348,0.088235,7.216365e-06,0.000048,0.000145,0.076923,0.058824,0.695652,Small
39841,24,11,high,moderate,moderate,0,0,0,1,0,0,0.304348,0.338235,2.159755e-04,0.000467,0.001830,0.153846,0.058824,0.130435,Middle
14273,10,3,high,low,moderate,0,0,0,0,0,0,0.217391,0.264706,4.809192e-04,0.000062,0.001004,0.076923,0.000000,0.782609,Small
35437,27,9,high,low,moderate,1,1,0,1,1,0,0.260870,0.279412,5.262792e-04,0.000073,0.000292,0.153846,0.058824,0.913043,Small
5864,24,4,high,low,high,1,1,0,1,0,0,0.260870,0.544118,3.427258e-03,0.002896,0.002514,0.076923,0.000000,0.347826,Large
3620,10,2,high,high,moderate,0,0,0,1,1,0,0.608696,0.191176,9.618383e-04,0.000087,0.000442,0.307692,0.000000,0.695652,Small
2734,23,8,high,low,high,0,0,0,1,0,0,0.173913,0.441176,2.407689e-03,0.000324,0.000687,0.000000,0.088235,0.260870,Middle
7399,24,7,moderate,high,low,0,1,0,1,0,0,0.565217,0.176471,3.402000e-03,0.000228,0.001822,0.538462,0.000000,0.000000,Middle
17314,10,6,moderate,moderate,low,0,0,0,1,1,0,0.347826,0.044118,5.224648e-03,0.000813,0.001083,0.076923,0.000000,0.739130,Middle
23858,10,8,high,high,low,1,1,0,1,1,0,0.521739,0.044118,1.623167e-03,0.000383,0.000515,0.153846,0.058824,0.086957,Middle


In [40]:
from apyori import apriori

In [41]:
data = []
columns = list(frames)
for k in columns:
    a = frames[k].values.tolist()
    for i in a:
        data.append(k +" is "+str(i))
len(data)

4080

In [45]:
this_rules = apriori(data, min_support=0.1, min_confidence=0.5)
#,min_lift=3, min_length=2)  
association_results = list(this_rules)
association_results

KeyboardInterrupt: 

In [136]:
def get_clean_rules(rules,output_col = "daily_view_classification is Large"):
    rules = [rule for rule in rules if output_col in rule.items]
    clean_rules = {}
    for rule in rules:
        support = rule.support
        # Select rules with right side containing output
        for r in [sb for sb in rule.ordered_statistics if (output_col in sb.items_add and len(sb.items_base) > 0)]:
            clean_rules["{} => {}".format(" + ".join(r.items_base), output_col)] = {"support": support, "confidence": r.confidence}            
    return(clean_rules)

In [137]:
Rules = get_clean_rules(this_rules,output_col = "daily_view_classification is Large") #get the clean rules

KeyboardInterrupt: 

[]